In [1]:
import sys
from pathlib import Path
import json

import pandas as pd
import numpy as np
import gtfstk as gt
import shapely.geometry as sg
import shapely.ops as so
import folium as fl

sys.path.append('../')

import make_gtfs as mg


DATA_DIR = Path('../data')

%load_ext autoreload
%autoreload 2


In [2]:
pfeed = mg.ProtoFeed(DATA_DIR/'auckland')


In [12]:

    
geo_stops = gt.geometrize_stops(pfeed.stops, use_utm=True)
linestring_by_shape = mg.build_linestring_by_shape(pfeed, use_utm=True)
linestring = linestring_by_shape['shp2']
nearby_stops = mg.get_nearby_stops(geo_stops, linestring, 'right').to_crs(gt.WGS84)
nearby_stops

,zone_id,stop_id,parent_station,stop_desc,stop_name,location_type,stop_code,geometry
11,merged_20,11053-20180426095657_v66.18,NaN,NaN,Ponsonby Rd and K'Rd,1,11053,POINT (174.75212 -36.8593)
17,merged_21,1086-20180426102952_v66.19,11717-20180426102952_v66.19,NaN,Karangahape Rd Outside Cemetery,0,1086,POINT (174.76281 -36.85820999999999)
40,merged_21,7190-20180426102952_v66.19,11503-20180426102952_v66.19,NaN,272 Parnell Rd,0,7190,POINT (174.78109 -36.85630999999999)
43,merged_20,11710-20180426095657_v66.18,NaN,NaN,Park Rd/Grafton Station,1,11710,POINT (174.77074 -36.86557)
108,merged_21,7197-20180426102952_v66.19,11507-20180426102952_v66.19,NaN,545 Parnell Rd,0,7197,POINT (174.78053 -36.86335)
121,merged_21,7222-20180426102952_v66.19,11045-20180426102952_v66.19,NaN,Opp 253 Ponsonby Rd,0,7222,POINT (174.744 -36.84931)
132,merged_21,11048-20180426102952_v66.19,NaN,NaN,Ponsonby Rd and Vermont,1,11048,POINT (174.74505 -36.85256999999999)
140,merged_21,7196-20180426102952_v66.19,11506-20180426102952_v66.19,NaN,470 Parnell Rd,0,7196,POINT (174.78229 -36.86149)
152,merged_20,7188-20180426095657_v66.18,11501-20180426095657_v66.18,NaN,128 Parnell Rd,0,7188,POINT (174.77921 -36.85331000000001)
176,merged_20,11505-20180426095657_v66.18,NaN,NaN,Parnell Rd and St Marys,1,11505,POINT (174.78279 -36.86005)


In [25]:
collection = json.loads(nearby_stops.to_json())
collection['features'].append(pfeed.shapes['features'][1])
center = list(nearby_stops.geometry.iat[0].coords[0])[::-1]
print(center)
my_map = fl.Map(location=center, zoom_start=13, tiles='cartodbpositron')

for f in collection['features']:
    prop = f['properties']

    # Add stop
    if f['geometry']['type'] == 'Point':
        lon, lat = f['geometry']['coordinates']
        fl.CircleMarker(
            location=[lat, lon],
            radius=8,
            fill=True,
            color='red',
            weight=1,
        ).add_to(my_map)

    # Add path
    else:
        prop['color'] = 'black'
        path = fl.GeoJson(f,
            name=prop['shape_id'],
            style_function=lambda x: {
              'color': x['properties']['color']},
        )
        path.add_to(my_map)

my_map

[-36.859300000000005, 174.75212000000002]


In [5]:
import shapely.wkt as sw
import shapely.ops as so

#s = sg.shape(pfeed.shapes['features'][0]['geometry'])
s = sw.loads("LINESTRING(0 0, 0.2 0.1, 0.3 0.6, 0.4 0.1, 1 1)")
#s = sg.LineString([(x, y) for x, y in reversed(s.coords)])

eps = 10e-3
b0 = s.buffer(eps, cap_style=3)
b1 = s.buffer(0.05, cap_style=2)
diff = b1.difference(b0)
left, right = list(so.polygonize(diff))


In [6]:
from matplotlib import pyplot
from shapely.geometry import LineString
from descartes.patch import PolygonPatch

fig = pyplot.figure(1, figsize=(10, 4), dpi=180)


ax = fig.add_subplot(121)
patch2a = PolygonPatch(left, facecolor='#cccccc', edgecolor='#999999')
ax.add_patch(patch2a)
patch2b = PolygonPatch(right, facecolor='#99ccff', edgecolor='#6699cc')
ax.add_patch(patch2b)

ax.set_xlim(-1, 2)
ax.set_ylim(-1, 2)

pyplot.show()

<Figure size 1800x720 with 1 Axes>